## Pytho-dotenv

In [37]:
import os
from dotenv import load_dotenv,find_dotenv
load_dotenv(find_dotenv(),override=True)

True

### Creating a simple tool : Circumference calculator

In [38]:
from langchain.tools import BaseTool
from math import pi
from typing import Union

# The class is inheriting from the BaseTool class
# BaseTool is the required template for a LangChain tool
# name and description are two attributes that langchain requires to recognize
# an object as a tool
class CircumferenceTool(BaseTool):
    
    name="Circumference Calculator"
    # describe when to use tool and when not to use tool
    description="use this tool when you need to calculate circumference of a circle using the radius"
    
    #Union[int,float] is used to specify that radius can either be a float or int
    def _run(self,radius: Union[int,float]):
        return float(radius)*2.0*pi
    

class AreaTool(BaseTool):
    
    name="Area Calculator"
    # describe when to use tool and when not to use tool
    description="use this tool when you need to calculate a Area\
     using radius of the circle"
    
    #Union[int,float] is used to specify that radius can either be a float or int
    def _run(self,radius: Union[int,float]):
        return (float(radius)**2)*pi

In [39]:
class FactorialTool(BaseTool):
    
    name = "Factorial Calculator"
    description = "Use this tool when you need to calculate the factorial of a number"
    
    def _run(self, number: int):
        
        number=int(number)
        if number < 0:
            raise ValueError("Factorial is not defined for negative numbers")
        result = 1
        for i in range(1, int(number) + 1):
            result *= i
        return result


In [40]:
from langchain.chat_models import ChatOpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

llm = ChatOpenAI( model_name='gpt-3.5-turbo',temperature = 0)

# k = 5 means chat history will store the previous 5 human-AI interactions
c_mem= ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=5,
    return_messages=True
    )

In [41]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI

In [42]:
from langchain.agents import initialize_agent

#we have to provide list of tools to the agent
tools = [CircumferenceTool(),AreaTool(),FactorialTool()]

# initialize agenst with tools

agent = initialize_agent(
    
    #agent type
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    tools=tools,
    llm=llm,
    verbose=True,
    max_iteration=7,
    early_stopping_method='generate'
)

In [26]:
agent("Calculate the factorial of number 6 and then use the answer as the radius for calculating the area and circumference of the circle")



> Entering new AgentExecutor chain...
To calculate the factorial of 6, I will use the Factorial Calculator. Then, I will use the result as the radius for calculating the area and circumference of the circle.
Action: Factorial Calculator
Action Input: 6
Observation: 720
Thought:Now that I have the factorial of 6, I can use it as the radius for calculating the area and circumference of the circle.
Action: Area Calculator
Action Input: 720
Observation: 1628601.6316209487
Thought:Now that I have the area of the circle, I can calculate the circumference using the same radius.
Action: Circumference Calculator
Action Input: 720
Observation: 4523.893421169302
Thought:

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-Zgvqg40aCEUeeAAUlnbvIafN on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-Zgvqg40aCEUeeAAUlnbvIafN on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit ht

I now know the final answer. The factorial of 6 is 720, and using that as the radius, the area of the circle is approximately 1628601.6316209487 and the circumference is approximately 4523.893421169302.
Final Answer: The factorial of 6 is 720, and the area of the circle with a radius of 720 is approximately 1628601.6316209487 and the circumference is approximately 4523.893421169302.

> Finished chain.


{'input': 'Calculate the factorial of number 6 and then use the answer as the radius for calculating the area and circumference of the circle',
 'output': 'The factorial of 6 is 720, and the area of the circle with a radius of 720 is approximately 1628601.6316209487 and the circumference is approximately 4523.893421169302.'}

In [27]:
print(agent.agent.llm_chain.prompt.template)

Answer the following questions as best you can. You have access to the following tools:

Circumference Calculator: use this tool when you need to calculate circumference of a circle using the radius
Area Calculator: use this tool when you need to calculate a Area     using radius of the circle
Factorial Calculator: Use this tool when you need to calculate the factorial of a number

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Circumference Calculator, Area Calculator, Factorial Calculator]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [21]:
agent("Who is Leonardo Di Capri")



> Entering new AgentExecutor chain...
I don't have access to any tools that can answer this question.
Final Answer: I don't know who Leonardo Di Capri is.

> Finished chain.


{'input': 'Who is Leonardo Di Capri',
 'output': "I don't know who Leonardo Di Capri is."}

In [ ]:
agent("can you calculate the area of a circle\
that has a radius of 7.81 mm")

In [ ]:
agent("Calculate the circumference of a circle\
that has a radius of 7.81")

* Final answer is the action the agent uses when it has completed its steps, which means the agent used the Circumference Calculator.

* If the agent choose not to use the circumference calculator, then change the SystemMessage of the agent

In [ ]:
# existing prompt
print(agent.agent.llm_chain.prompt.messages[0].prompt.template)

In [28]:
sys_msg = """Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Unfortunately, Assistant is terrible at maths. When provided with math questions, no matter how simple, assistant always refers to it's trusty tools and absolutely does NOT try to answer math questions by itself.

To solve problems related to Pythagorean theorem, the Assistant must use the PythagorusTool available in the tools list.

Overall, Assistant is a powerful system that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assistant is here to assist.
"""

In [32]:
new_prompt = agent.agent.create_prompt(
    system_message=sys_msg,
    tools=tools
)

agent.agent.llm_chain.prompt = new_prompt

TypeError: create_prompt() got an unexpected keyword argument 'system_message'

In [33]:
agent("can you calculate the circumference of a circle that has a radius of 7")



> Entering new AgentExecutor chain...
To calculate the circumference of a circle, I need to use the Circumference Calculator tool.
Action: Circumference Calculator
Action Input: Radius = 7

ValueError: could not convert string to float: 'Radius = 7'

### Tools with multiple parameters

* Now, we'll create a tool to calculate the hypotenuse of a triangle from a give set of two sides/or an angle.

In [43]:
from langchain.tools import BaseTool
from typing import Optional
from math import sqrt,cos,sin

desc = (
    "use this tool when you need to calculate the length of a hypotenuse"
    "given one or two sides of a triangle and/or an angle (in degrees). "
    "To use the tool, you must provide at least two of the following parameters "
    "['adjacent_side', 'opposite_side', 'angle']."
)

class PythagorasTool(BaseTool):
    
    name='Hypotenuse Calculator'
    description=desc
    
    def _run(
        self,
        adjacent_side: Optional[Union[int, float]] = None,
        opposite_side: Optional[Union[int, float]] = None,
        angle: Optional[Union[int, float]] = None
    ):
        # check for the values we have been given
        if adjacent_side and opposite_side:
            return sqrt(float(adjacent_side)**2 + float(opposite_side)**2)
        elif adjacent_side and angle:
            return adjacent_side / cos(float(angle))
        elif opposite_side and angle:
            return opposite_side / sin(float(angle))
        else:
            return "Could not calculate the hypotenuse of the triangle. Need two or more of `adjacent_side`, `opposite_side`, or `angle`."


In [44]:
agent.tools.append(PythagorasTool())

In [45]:
agent("If I have a triangle with adjacent side and opposite side of lengths 51 and 34 respectively,\
what is the length of the hypotenuse?")



> Entering new AgentExecutor chain...


OutputParserException: Could not parse LLM output: `This is a right triangle, so I can use the Pythagorean theorem to find the length of the hypotenuse.
Action: None`

In [ ]:
# existing prompt
print(agent.agent.llm_chain.prompt.messages[0].prompt.template)